# **Visualization of the NTU RGB+D Action Recognition Dataset**

___
___

## **1. Initialization**

### *1.1. Imports*

In [1]:
from PIL import Image, ImageDraw
from scipy.spatial.transform import Rotation as R
from tqdm import tqdm

import matplotlib.pyplot as plt
import numpy as np
import os

from torch.utils.data import Dataset

import torch

In [4]:
# to fill the requirement.txt file we use the following line of code:
# import session_info
# session_info.show()

In [5]:
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("device: {}".format(device))

device: cuda


### *1.2. Data Loading*

In [6]:
data2D_dir = "data/nturgbd60_skeletons_2D/"
data3D_dir = "data/nturgbd60_skeletons_3D/"

data2D_files = os.listdir(data2D_dir)
data3D_files = os.listdir(data3D_dir)

In [7]:
with open("data/actions.txt", 'r') as actions_file:
    actions = [line.replace('\n', '') for line in actions_file.readlines()]
    actions_file.close()

In [46]:
classes = [i for i in range(60)]
# for i,elem in enumerate(classes):
#    print("class {} : {}".format(i, actions[elem]))

In [47]:
class HumanActionDataset2D(Dataset):

    def __init__(self, data_dir, data_files, classes, flatten=False):
        self.data_dir = data_dir
        self.data_files = [data_file for data_file in data_files if int(data_file[17:-4])-1 in classes]
        self.classes = classes
        self.flatten =  flatten

    def __len__(self):
        return len(self.data_files)

    def __getitem__(self, idx):
        tensor = torch.Tensor(np.load(self.data_dir + self.data_files[idx]))
        tensor = tensor.reshape((tensor.shape[0], tensor.shape[1]*tensor.shape[2]))
        label = self.classes.index(int(self.data_files[idx][17:-4])-1)
        return (tensor, label)

In [48]:
class HumanActionDataset3D(Dataset):

    def __init__(self, data_dir, data_files, classes, flatten=False):
        self.data_dir = data_dir
        self.data_files = [data_file for data_file in data_files if int(data_file[17:-4])-1 in classes]
        self.classes = classes
        self.flatten =  flatten

    def __len__(self):
        return len(self.data_files)

    def __getitem__(self, idx):
        tensor = torch.Tensor(np.load(self.data_dir + self.data_files[idx]))
        tensor = tensor.reshape((tensor.shape[0], tensor.shape[1]*tensor.shape[2]))
        label = self.classes.index(int(self.data_files[idx][17:-4])-1)
        return (tensor, label)

In [49]:
HAD2D = HumanActionDataset2D(data2D_dir, data2D_files, classes)
HAD3D = HumanActionDataset3D(data3D_dir, data3D_files, classes)

___

## **2. Create Images**

Reminder:

![skeleton](./assets/skeleton.png)

In [50]:
# sequences of joints to display the members of the squeletons (add +1 to find the corresponding joints in the above graphic)

bust_joints = [0, 1, 20, 2, 3]
arm_joints = [23, 24, 11, 10, 9, 8, 20, 4, 5, 6, 7, 22, 21]
leg_joints = [19, 18, 17, 16, 0, 12, 13, 14, 15]

body_parts = [bust_joints, arm_joints, leg_joints]

In [51]:
def get_image_sequence(i, dataset2D=HAD2D, dataset3D=HAD3D):
    
    """
    inputs:
    * (int) i : index of the sample to consider
    * dataset2D : dataset with the pixel information
    * dataset3D : dataset with the space information (if not None we will consider the depth in the generated images)

    output:
    * output : sequence of images with, eventually, depth
    """
    
    tensor2D = dataset2D[i][0]
    output = np.zeros((tensor2D.shape[0],25,3))
    z_sequence = np.zeros((tensor2D.shape[0],25))

    if dataset3D != None:
        tensor3D = dataset3D[i][0]
        z_sequence = np.array([[tensor3D[k,3*i+2] for i in range(25)] for k in range(tensor3D.shape[0])])     
    
    for m in range(tensor2D.shape[0]):
        for n in range(25):
            x = tensor2D[m,2*n].item()
            y = tensor2D[m,2*n+1].item()
            x = min(max(0,int(x)),1920)-1
            y = min(max(0,int(y)),1080)-1
            z = z_sequence[m,n].item()
            output[m,n] = [x,y,z]
    
    return output

In [52]:
def get_image_sequence_normalize(i, dataset2D=HAD2D, dataset3D=HAD3D):
    """
    get sequence of images with, eventually, depth but also normalize the depth between 0 and 1
    """
    image_sequence = get_image_sequence(i, dataset2D, dataset3D)
    mini = image_sequence[:,:,2].min()
    maxi = image_sequence[:,:,2].max()
    if mini != maxi:
        image_sequence[:,:,2] = (image_sequence[:,:,2]-mini)/(maxi-mini)
    return image_sequence

In [53]:
def create_image(image_frame):

    img = Image.new("RGB", (1920,1080 ), color="black")
    for body_part in body_parts:
        for i in range(len(body_part)-1):
            a = image_frame[body_part[i]]
            b = image_frame[body_part[i+1]]-a
            n = 10
            line = np.array([a+(i/n)*b for i in range(n)])
            for i in range(len(line)-1):
                x1,y1,z = line[i]
                x2,y2,_ = line[i+1]
                draw = ImageDraw.Draw(img)
                color = tuple((int(255*z),0,int(255*(1-z))))
                draw.line((x1,y1,x2,y2), fill=color,width=5)
    return img

In [54]:
# example (first element of the dataset)
create_image(get_image_sequence_normalize(15)[0,:,:])

In [55]:
def create_gif(i, filepath, dataset2D=HAD2D, dataset3D=HAD3D, normalize=True):
    if normalize:
        image_sequence = get_image_sequence_normalize(i, dataset2D, dataset3D)
    else:
        image_sequence = get_image_sequence(i, dataset2D, dataset3D)
    seq = [create_image(frame) for frame in image_sequence]
    seq[0].save(filepath, save_all=True, append_images=seq[1:])

In [56]:
if "gifs" not in os.listdir("./data/"):
    os.mkdir("./data/gifs")

In [58]:
for index in tqdm(range(100)):
    name = data2D_files[index][:-4]
    if name+".gif" not in os.listdir("./data/gifs"):
        create_gif(index, "./data/gifs/{}.gif".format(name))

100%|██████████| 100/100 [01:16<00:00,  1.32it/s]


___
___